In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
import sys
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Load the data
training = pd.read_csv('training_data.csv', sep=',')
test = pd.read_csv('songs_to_classify.csv', sep=',')

only use five features version

In [3]:
# select which features to use
features = ['danceability','key','loudness','instrumentalness','liveness']

# i learn how to do one-hot encode from chatgpt because i realized there is no size meaning in the key feature.  
encoder = OneHotEncoder(sparse=False)

X_train_selected = training[features]
X_test_selected = test[features]
# OneHotEncode the 'key' feature 
X_train_key_encoded = encoder.fit_transform(X_train_selected[['key']])
X_test_key_encoded = encoder.transform(X_test_selected[['key']])

X_train_no_key = X_train_selected.drop('key', axis=1).values
X_test_no_key = X_test_selected.drop('key', axis=1).values

X_train = np.hstack([X_train_no_key, X_train_key_encoded])
X_test = np.hstack([X_test_no_key, X_test_key_encoded])

# extract labels
y_train = training.loc[:, 'label'].values

In [4]:
# Normalize data

X_trainn = X_train * 1 / np.max(np.abs(X_train), axis=0)
X_testn = X_test * 1 / np.max(np.abs(X_test), axis=0)


using all features version

In [5]:
# i learned how to choose all the features from chatgpt but now i think i could figure it out myself

#encoder = OneHotEncoder(sparse=False)
#X_train_key_encoded = encoder.fit_transform(X_train_selected[['key']])
#X_test_key_encoded = encoder.transform(X_test_selected[['key']])

# Remove the 'key' column and keep other features
#X_train_no_key_time = training.loc[:, training.columns != 'key'].drop('label', axis=1).values
#X_test_no_key_time = test.loc[:, test.columns != 'key'].values

# Merge OneHot encoded 'key' with other features
#X_train = np.hstack([X_train_no_key_time, X_train_key_encoded, X_train_time_signature_encoded])
#X_test = np.hstack([X_test_no_key_time, X_test_key_encoded, X_test_time_signature_encoded])

# Extract labels
#y_train = training.loc[:, 'label'].values

In [6]:
# Define the random forest model
rf_model = RandomForestClassifier()

In [7]:
# Define the parameter grid for hyperparameter tuning
# i use chatgpt to generate this code for choosing these hyperparameters using GridSearchCV. 
# I know this method but i was not sure how to implement it in code.
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]      # Minimum number of samples required to be at a leaf node
}

# Initialize GridSearchCV
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                               scoring='accuracy', cv=5)  # 5-fold cross-validation

# Train the model using GridSearchCV
grid_search_rf.fit(X=X_trainn, y=y_train)

# Get the best parameters
best_params_rf = grid_search_rf.best_params_
print(f'Best parameters for Random Forest: {best_params_rf}')


Best parameters for Random Forest: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}


In [8]:
# Make predictions with the best model
best_rf_model = grid_search_rf.best_estimator_
# Chagpt designed this output format for me and i asked it to remove spaces between elements so that i could use it for submission
predictions_rf = best_rf_model.predict(X=X_testn).reshape(-1, 1).astype(int).reshape(1, -1)
#print(predictions_rf)
# Convert an array to a string and remove spaces between elements
print(''.join(map(str, predictions_rf.flatten())))

00010001001101001011001010000010111111010001100110001101100011000111100001010110110101100000001111110010000011110010110001101110100111111101101110001011101111111101100100101000001001111100100000100111


Cross Validation

In [9]:
# i use chatgpt to do cross validation because I need a standard to evaluate the pros and cons of different models
cv_scores_rf = cross_val_score(best_rf_model, X_trainn, y_train, cv=5, scoring='accuracy')
print(f'Random Forest cross-validation accuracy: {cv_scores_rf.mean():.4f} ± {cv_scores_rf.std():.4f}')

Random Forest cross-validation accuracy: 0.7440 ± 0.0288
